# YOLOOO

In [1]:
import os
import yaml

# --- PARAMETER KONFIGURASI YAML ---
# Jumlah kelas objek (HANYA kelas objek, tidak termasuk background)
NUM_CLASSES = 1

# Daftar nama kelas Anda
class_names = ['sakit']

# Path ke direktori data yang sudah Anda buat di /kaggle/working/
YOLO_DATA_PATH = "/kaggle/input/data-augmented-citra-jagung/YOLOtraining/Split_aug"

# Nama file konfigurasi
YAML_FILE_NAME = "data2.yaml"

# 1. Definisikan isi file YAML
data_yaml_content = {
    'path': YOLO_DATA_PATH,
    'train': 'train/images',  # Path relatif dari YOLO_DATA_PATH
    'val': 'val/images',      # Path relatif dari YOLO_DATA_PATH
    'test': 'test/images',    # Path relatif dari YOLO_DATA_PATH
    'nc': NUM_CLASSES,        # Jumlah kelas
    'names': class_names      # Nama-nama kelas
}

# 2. Tulis data ke file YAML di working directory
with open(os.path.join("/kaggle/working/", YAML_FILE_NAME), 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print(f"File konfigurasi {YAML_FILE_NAME} berhasil dibuat")
print(data_yaml_content)

File konfigurasi data2.yaml berhasil dibuat
{'path': '/kaggle/input/data-augmented-citra-jagung/YOLOtraining/Split_aug', 'train': 'train/images', 'val': 'val/images', 'test': 'test/images', 'nc': 1, 'names': ['sakit']}


Traing Model YOLO, beberapa cell berisikan hyperparameter yang berbeda-beda. Lokasi penyimpanan hasil dapat disesuaikan kembali.

In [ ]:
import os
import pandas as pd
from ultralytics import YOLO
import datetime # Import untuk unique sheet names

# --- KONFIGURASI PATH DAN HYPERPARAMETER ---

# Path ke file data.yaml Anda
DATA_YAML_PATH = "/kaggle/working/data2.yaml"

# Ganti dengan hyperparameter terbaik hasil tuning manual Anda (atau gunakan default)
BEST_LR = 0.01      # Learning Rate awal (Default)
BEST_MOMENTUM = 0.937 # Momentum (Default)
FINAL_EPOCHS = 20   # Jumlah epoch untuk pelatihan final (Ganti sesuai kebutuhan)
BATCH_SIZE = 32        # Ukuran batch (Sesuaikan dengan VRAM GPU Kaggle)

# Output directory di Google Drive untuk semua eksperimen YOLO
# Ini akan membuat folder seperti: /content/drive/MyDrive/EAS VISIKOM/YOLO_Output/yolo_production/YOLOv8_Final_Training_Model2
OUTPUT_BASE_DRIVE_DIR = "/kaggle/working/Hasil"

# Nama proyek (subfolder di OUTPUT_BASE_DRIVE_DIR)
PROJECT_NAME = "yolo_production"

# Nama eksperimen final (output akan disimpan di PROJECT_NAME/EXPERIMENT_NAME)
EXPERIMENT_NAME = 'YOLOv8_Final_Training_Model1'

# --- INIALISASI MODEL ---
# Muat model pre-trained (yolov8n.pt)
model = YOLO('yolov8n.pt')

# ----------------------------------------------------
# Bagian 1: Pelatihan Final
# ----------------------------------------------------
print("---> Memulai Pelatihan Final YOLOv8 --->")


TRAINING_ARGS = {
    'data': DATA_YAML_PATH,
    'epochs': FINAL_EPOCHS,
    'imgsz': 640,
    'batch': BATCH_SIZE,
    'name': EXPERIMENT_NAME, # Experiment name within the project
    'project': os.path.join(OUTPUT_BASE_DRIVE_DIR, PROJECT_NAME), # Full path to the project folder on Drive
    'lr0': BEST_LR,
    'momentum': BEST_MOMENTUM,
    'val': True, # Wajib untuk mendapatkan metrik validasi
    'plots': True # Menghasilkan plot loss dan metrik
}

# Mulai Pelatihan
results = model.train(
    **TRAINING_ARGS
)

print("\n✅ Pelatihan Selesai!")

# ----------------------------------------------------
# Bagian 2: Evaluasi dan Pencatatan Loss
# ----------------------------------------------------

# Tentukan path tempat hasil disimpan (otomatis dibuat oleh Ultralytics)
# Ultralytics menyimpan hasil relatif terhadap direktori kerja saat ini,
# biasanya di `runs/detect/project_name/experiment_name`.
# Objek `results` secara langsung menyediakan direktori penyimpanan.
OUTPUT_BASE_DIR = str(results.save_dir) # This will now be the Drive path
BEST_MODEL_PATH = os.path.join(OUTPUT_BASE_DIR, 'weights', 'best.pt')
RESULTS_CSV_PATH = os.path.join(OUTPUT_BASE_DIR, 'results.csv')

# Path untuk menyimpan hasil ke file Excel
# The Excel report should also go into the new OUTPUT_BASE_DIR
EXCEL_OUTPUT_DIR = os.path.join(OUTPUT_BASE_DIR, 'Excel_Reports') # Changed this line
os.makedirs(EXCEL_OUTPUT_DIR, exist_ok=True)
EXCEL_REPORT_PATH = os.path.join(EXCEL_OUTPUT_DIR, f'{PROJECT_NAME}_{EXPERIMENT_NAME}_training_results.xlsx') # Added experiment name to differentiate reports

print("\n--- Ringkasan Hasil Pelatihan ---")

# A. Mengambil Metrik mAP (Mean Average Precision)
try:
    # Akses metrik langsung dari objek results.metrics (DetMetrics object)
    map50 = results.metrics.box.map50
    map50_95 = results.metrics.box.map

    print(f"Metrik Deteksi (Bounding Box - B):")
    print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
    print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")

except Exception as e:
    print(f"Terjadi error saat mencoba mengakses mAP dari results.metrics: {e}. Mencoba membaca dari CSV...")
    # Fallback to reading from CSV if direct access fails or for consistency
    try:
        df_results_full = pd.read_csv(RESULTS_CSV_PATH)
        last_row_full = df_results_full.iloc[-1]
        map50 = last_row_full['metrics/mAP50(B)']
        map50_95 = last_row_full['metrics/mAP50-95(B)']
        print(f"Metrik Deteksi (Bounding Box - B) dari CSV:")
        print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
        print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")
    except Exception as csv_e:
        print(f"❌ ERROR membaca mAP dari CSV: {csv_e}")

# B. Mengambil Nilai Loss dari File results.csv dan menyimpannya ke Excel
try:
    # Membaca file results.csv yang dibuat oleh Ultralytics
    df_results = pd.read_csv(RESULTS_CSV_PATH)

    # Mencari baris terakhir (epoch terakhir)
    last_row = df_results.iloc[-1]

    # Kolom Loss di Ultralytics:
    # box_loss: Loss untuk bounding box localization
    # cls_loss: Loss untuk classification
    # dfl_loss: Loss untuk Distribution Focal Loss (digunakan di YOLOv8)

    train_box_loss = last_row['train/box_loss']
    train_cls_loss = last_row['train/cls_loss']
    val_box_loss = last_row['val/box_loss']
    val_cls_loss = last_row['val/cls_loss']

    print("\nNilai Loss di Akhir Pelatihan:")
    print(f"  Total Train Loss (Box+Cls): {(train_box_loss + train_cls_loss):.4f}")
    print(f"  Total Validasi Loss (Box+Cls): {(val_box_loss + val_cls_loss):.4f}")
    print(f"  (Detail: Train Box Loss: {train_box_loss:.4f} | Val Box Loss: {val_box_loss:.4f})")

    # --- Menyimpan ke Excel/Spreadsheet --- #
    # Buat nama sheet yang unik berdasarkan waktu
    timestamp_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    sheet_name = f"Run_{timestamp_str}"

    # Gunakan pd.ExcelWriter untuk menulis ke file Excel.
    # 'mode='a'' untuk append jika file sudah ada.
    # 'if_sheet_exists='new'' akan membuat sheet baru jika nama sheet sudah ada (atau file belum ada).
    # 'engine='openpyxl'' diperlukan untuk mode append dan .xlsx.
    with pd.ExcelWriter(EXCEL_REPORT_PATH, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df_results.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Hasil pelatihan disimpan ke sheet '{sheet_name}' di file Excel: {EXCEL_REPORT_PATH}")


except FileNotFoundError:
    print(f"\n❌ ERROR: File results.csv tidak ditemukan di {RESULTS_CSV_PATH}.")
    print("Pastikan eksperimen berjalan sepenuhnya dan tidak ada error file system.")
except Exception as e:
    print(f"\n❌ ERROR membaca Loss dari CSV atau menulis ke Excel: {e}")

# ----------------------------------------------------
# Bagian 3: Konfirmasi Penyimpanan Model
# ----------------------------------------------------
print("\n--- Konfirmasi Penyimpanan Model ---")
print(f"Model terbaik tersimpan secara otomatis di:")
print(f"💾 {BEST_MODEL_PATH}")
print(f"Log pelatihan (loss dan metrik) tersimpan di:")
print(f"📄 {RESULTS_CSV_PATH}")
print(f"Laporan Excel tersimpan di:")
print(f"📊 {EXCEL_REPORT_PATH}")


In [5]:
import os
import pandas as pd
from ultralytics import YOLO
import datetime

# --- KONFIGURASI PATH DAN HYPERPARAMETER ---

# Path ke file data.yaml Anda
DATA_YAML_PATH = "/kaggle/working/data2.yaml"

# Hyperparameter
BEST_LR = 0.01      
BEST_MOMENTUM = 0.937 
FINAL_EPOCHS = 100   
BATCH_SIZE = 16       

# Output directory (Di Kaggle Output)
OUTPUT_BASE_DRIVE_DIR = "/kaggle/working/Hasil"
PROJECT_NAME = "yolo_production"
EXPERIMENT_NAME = 'YOLOv8_Final_Training_Model_final'

# --- INIALISASI MODEL ---
model = YOLO('yolov8n.pt')

# ----------------------------------------------------
# Bagian 1: Pelatihan Final
# ----------------------------------------------------
print("---> Memulai Pelatihan Final YOLOv8 --->")

TRAINING_ARGS = {
    'data': DATA_YAML_PATH,
    'epochs': FINAL_EPOCHS,
    'imgsz': 640,
    'batch': BATCH_SIZE,
    'name': EXPERIMENT_NAME, 
    'project': os.path.join(OUTPUT_BASE_DRIVE_DIR, PROJECT_NAME), 
    'lr0': BEST_LR,
    'momentum': BEST_MOMENTUM,
    'val': True, 
    'plots': True 
}

# Mulai Pelatihan
results = model.train(**TRAINING_ARGS)

print("\n✅ Pelatihan Selesai!")

# ----------------------------------------------------
# Bagian 2: Evaluasi dan Pencatatan Loss
# ----------------------------------------------------

# Konversi path ke string agar aman
OUTPUT_BASE_DIR = str(results.save_dir) 
BEST_MODEL_PATH = os.path.join(OUTPUT_BASE_DIR, 'weights', 'best.pt')
RESULTS_CSV_PATH = os.path.join(OUTPUT_BASE_DIR, 'results.csv')

# Setup Folder Excel
EXCEL_OUTPUT_DIR = os.path.join(OUTPUT_BASE_DIR, 'Excel_Reports') 
os.makedirs(EXCEL_OUTPUT_DIR, exist_ok=True)
EXCEL_REPORT_PATH = os.path.join(EXCEL_OUTPUT_DIR, f'{PROJECT_NAME}_{EXPERIMENT_NAME}_training_results.xlsx') 

print("\n--- Ringkasan Hasil Pelatihan ---")

# A. Mengambil Metrik mAP
# Menggunakan logic fallback yang Anda buat (sudah bagus)
try:
    map50 = results.box.map50 # Update syntax terbaru Ultralytics biasanya results.box.map50
    map50_95 = results.box.map
    print(f"Metrik Deteksi (Bounding Box - B):")
    print(f"  mAP50 : {map50:.4f}")
    print(f"  mAP50-95 : {map50_95:.4f}")
except:
    try:
        # Fallback ke metrics object lama atau CSV
        map50 = results.metrics.box.map50
        map50_95 = results.metrics.box.map
        print(f"Metrik Deteksi (Bounding Box - B) [via metrics object]:")
        print(f"  mAP50 : {map50:.4f}")
        print(f"  mAP50-95 : {map50_95:.4f}")
    except Exception as e:
        print(f"Gagal akses langsung metrik object, membaca dari CSV... ({e})")
        try:
            df_results_full = pd.read_csv(RESULTS_CSV_PATH)
            # Menghapus spasi di nama kolom agar aman
            df_results_full.columns = df_results_full.columns.str.strip()
            last_row_full = df_results_full.iloc[-1]
            map50 = last_row_full['metrics/mAP50(B)']
            map50_95 = last_row_full['metrics/mAP50-95(B)']
            print(f"Metrik Deteksi dari CSV: mAP50={map50:.4f}, mAP50-95={map50_95:.4f}")
        except Exception as csv_e:
            print(f"❌ ERROR membaca mAP dari CSV: {csv_e}")

# B. Simpan ke Excel (BAGIAN YANG DIPERBAIKI)
try:
    df_results = pd.read_csv(RESULTS_CSV_PATH)
    
    # Bersihkan nama kolom (kadang ada spasi ekstra di CSV output YOLO)
    df_results.columns = df_results.columns.str.strip()

    timestamp_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    sheet_name = f"Run_{timestamp_str}"

    print(f"Menyimpan ke Excel: {EXCEL_REPORT_PATH}")

    # LOGIKA PERBAIKAN: Cek apakah file sudah ada atau belum
    if os.path.exists(EXCEL_REPORT_PATH):
        # Jika file ADA, gunakan mode 'a' (append) dan if_sheet_exists='new'
        with pd.ExcelWriter(EXCEL_REPORT_PATH, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
            df_results.to_excel(writer, sheet_name=sheet_name, index=False)
        print("Mode: APPEND (File sudah ada)")
    else:
        # Jika file TIDAK ADA, gunakan mode 'w' (write)
        # PENTING: Jangan gunakan parameter 'if_sheet_exists' di mode 'w'
        with pd.ExcelWriter(EXCEL_REPORT_PATH, engine='openpyxl', mode='w') as writer:
            df_results.to_excel(writer, sheet_name=sheet_name, index=False)
        print("Mode: WRITE (File baru dibuat)")

    print(f"✅ Hasil pelatihan berhasil disimpan ke sheet '{sheet_name}'")

except Exception as e:
    print(f"\n❌ ERROR Manajemen Excel: {e}")

# ----------------------------------------------------
# Bagian 3: Konfirmasi
# ----------------------------------------------------
print("\n--- Selesai ---")
print(f"Model: {BEST_MODEL_PATH}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
---> Memulai Pelatihan Final YOLOv8 --->
Ultralytics 8.3.240 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data2.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,

Corrupt JPEG data: 67561 extraneous bytes before marker 0xd5
Corrupt JPEG data: 61255 extraneous bytes before marker 0xd5


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      2.15G      2.217       3.25      1.866         19        640: 100% ━━━━━━━━━━━━ 21/21 1.2it/s 17.3s0.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 27% ━━━───────── 3/11 1.5it/s 2.5s<5.5s

Corrupt JPEG data: 61255 extraneous bytes before marker 0xd5


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 91% ━━━━━━━━━━╸─ 10/11 2.6it/s 9.9s<0.4s

...


100 epochs completed in 0.645 hours.
Optimizer stripped from /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final/weights/last.pt, 6.3MB
Optimizer stripped from /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final/weights/best.pt, 6.3MB

Validating /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final/weights/best.pt...
Ultralytics 8.3.240 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 73% ━━━━━━━━╸─── 8/11 2.6it/s 13.1s<1.1s

Corrupt JPEG data: 67561 extraneous bytes before marker 0xd5


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 1.7s/it 18.7s0.4s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        346       2909      0.524      0.373      0.393      0.167
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final

✅ Pelatihan Selesai!

--- Ringkasan Hasil Pelatihan ---
Metrik Deteksi (Bounding Box - B):
  mAP50 : 0.3931
  mAP50-95 : 0.1666
Menyimpan ke Excel: /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final/Excel_Reports/yolo_production_YOLOv8_Final_Training_Model_final_training_results.xlsx
Mode: WRITE (File baru dibuat)
✅ Hasil pelatihan berhasil disimpan ke sheet 'Run_20251218_071138'

--- Selesai ---
Model: /kaggle/working/Hasil/yolo_production/YOLOv8_Final_Training_Model_final/weights/best.pt


In [ ]:
import os
import pandas as pd
from ultralytics import YOLO
import datetime # Import untuk unique sheet names

# --- KONFIGURASI PATH DAN HYPERPARAMETER ---

# Path ke file data.yaml Anda
DATA_YAML_PATH = "/kaggle/working/data2.yaml"

# Ganti dengan hyperparameter terbaik hasil tuning manual Anda (atau gunakan default)
BEST_LR = 0.005      # Learning Rate awal (Default)
BEST_MOMENTUM = 0.937 # Momentum (Default)
FINAL_EPOCHS = 20   # Jumlah epoch untuk pelatihan final (Ganti sesuai kebutuhan)
BATCH_SIZE = 32        # Ukuran batch (Sesuaikan dengan VRAM GPU Kaggle)

# Output directory di Google Drive untuk semua eksperimen YOLO
# Ini akan membuat folder seperti: /content/drive/MyDrive/EAS VISIKOM/YOLO_Output/yolo_production/YOLOv8_Final_Training_Model2
OUTPUT_BASE_DRIVE_DIR = "/kaggle/working/Hasi3"

# Nama proyek (subfolder di OUTPUT_BASE_DRIVE_DIR)
PROJECT_NAME = "yolo_production"

# Nama eksperimen final (output akan disimpan di PROJECT_NAME/EXPERIMENT_NAME)
EXPERIMENT_NAME = 'YOLOv8_Final_Training_Model4'

# --- INIALISASI MODEL ---
# Muat model pre-trained (yolov8n.pt)
model = YOLO('yolov8n.pt')

# ----------------------------------------------------
# Bagian 1: Pelatihan Final
# ----------------------------------------------------
print("---> Memulai Pelatihan Final YOLOv8 --->")


TRAINING_ARGS = {
    'data': DATA_YAML_PATH,
    'epochs': FINAL_EPOCHS,
    'imgsz': 640,
    'batch': BATCH_SIZE,
    'name': EXPERIMENT_NAME, # Experiment name within the project
    'project': os.path.join(OUTPUT_BASE_DRIVE_DIR, PROJECT_NAME), # Full path to the project folder on Drive
    'lr0': BEST_LR,
    'momentum': BEST_MOMENTUM,
    'val': True, # Wajib untuk mendapatkan metrik validasi
    'plots': True # Menghasilkan plot loss dan metrik
}

# Mulai Pelatihan
results = model.train(
    **TRAINING_ARGS
)

print("\n✅ Pelatihan Selesai!")

# ----------------------------------------------------
# Bagian 2: Evaluasi dan Pencatatan Loss
# ----------------------------------------------------

# Tentukan path tempat hasil disimpan (otomatis dibuat oleh Ultralytics)
# Ultralytics menyimpan hasil relatif terhadap direktori kerja saat ini,
# biasanya di `runs/detect/project_name/experiment_name`.
# Objek `results` secara langsung menyediakan direktori penyimpanan.
OUTPUT_BASE_DIR = str(results.save_dir) # This will now be the Drive path
BEST_MODEL_PATH = os.path.join(OUTPUT_BASE_DIR, 'weights', 'best.pt')
RESULTS_CSV_PATH = os.path.join(OUTPUT_BASE_DIR, 'results.csv')

# Path untuk menyimpan hasil ke file Excel
# The Excel report should also go into the new OUTPUT_BASE_DIR
EXCEL_OUTPUT_DIR = os.path.join(OUTPUT_BASE_DIR, 'Excel_Reports') # Changed this line
os.makedirs(EXCEL_OUTPUT_DIR, exist_ok=True)
EXCEL_REPORT_PATH = os.path.join(EXCEL_OUTPUT_DIR, f'{PROJECT_NAME}_{EXPERIMENT_NAME}_training_results.xlsx') # Added experiment name to differentiate reports

print("\n--- Ringkasan Hasil Pelatihan ---")

# A. Mengambil Metrik mAP (Mean Average Precision)
try:
    # Akses metrik langsung dari objek results.metrics (DetMetrics object)
    map50 = results.metrics.box.map50
    map50_95 = results.metrics.box.map

    print(f"Metrik Deteksi (Bounding Box - B):")
    print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
    print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")

except Exception as e:
    print(f"Terjadi error saat mencoba mengakses mAP dari results.metrics: {e}. Mencoba membaca dari CSV...")
    # Fallback to reading from CSV if direct access fails or for consistency
    try:
        df_results_full = pd.read_csv(RESULTS_CSV_PATH)
        last_row_full = df_results_full.iloc[-1]
        map50 = last_row_full['metrics/mAP50(B)']
        map50_95 = last_row_full['metrics/mAP50-95(B)']
        print(f"Metrik Deteksi (Bounding Box - B) dari CSV:")
        print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
        print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")
    except Exception as csv_e:
        print(f"❌ ERROR membaca mAP dari CSV: {csv_e}")

# B. Mengambil Nilai Loss dari File results.csv dan menyimpannya ke Excel
try:
    # Membaca file results.csv yang dibuat oleh Ultralytics
    df_results = pd.read_csv(RESULTS_CSV_PATH)

    # Mencari baris terakhir (epoch terakhir)
    last_row = df_results.iloc[-1]

    # Kolom Loss di Ultralytics:
    # box_loss: Loss untuk bounding box localization
    # cls_loss: Loss untuk classification
    # dfl_loss: Loss untuk Distribution Focal Loss (digunakan di YOLOv8)

    train_box_loss = last_row['train/box_loss']
    train_cls_loss = last_row['train/cls_loss']
    val_box_loss = last_row['val/box_loss']
    val_cls_loss = last_row['val/cls_loss']

    print("\nNilai Loss di Akhir Pelatihan:")
    print(f"  Total Train Loss (Box+Cls): {(train_box_loss + train_cls_loss):.4f}")
    print(f"  Total Validasi Loss (Box+Cls): {(val_box_loss + val_cls_loss):.4f}")
    print(f"  (Detail: Train Box Loss: {train_box_loss:.4f} | Val Box Loss: {val_box_loss:.4f})")

    # --- Menyimpan ke Excel/Spreadsheet --- #
    # Buat nama sheet yang unik berdasarkan waktu
    timestamp_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    sheet_name = f"Run_{timestamp_str}"

    # Gunakan pd.ExcelWriter untuk menulis ke file Excel.
    # 'mode='a'' untuk append jika file sudah ada.
    # 'if_sheet_exists='new'' akan membuat sheet baru jika nama sheet sudah ada (atau file belum ada).
    # 'engine='openpyxl'' diperlukan untuk mode append dan .xlsx.
    with pd.ExcelWriter(EXCEL_REPORT_PATH, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df_results.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Hasil pelatihan disimpan ke sheet '{sheet_name}' di file Excel: {EXCEL_REPORT_PATH}")


except FileNotFoundError:
    print(f"\n❌ ERROR: File results.csv tidak ditemukan di {RESULTS_CSV_PATH}.")
    print("Pastikan eksperimen berjalan sepenuhnya dan tidak ada error file system.")
except Exception as e:
    print(f"\n❌ ERROR membaca Loss dari CSV atau menulis ke Excel: {e}")

# ----------------------------------------------------
# Bagian 3: Konfirmasi Penyimpanan Model
# ----------------------------------------------------
print("\n--- Konfirmasi Penyimpanan Model ---")
print(f"Model terbaik tersimpan secara otomatis di:")
print(f"💾 {BEST_MODEL_PATH}")
print(f"Log pelatihan (loss dan metrik) tersimpan di:")
print(f"📄 {RESULTS_CSV_PATH}")
print(f"Laporan Excel tersimpan di:")
print(f"📊 {EXCEL_REPORT_PATH}")


In [ ]:
import os
import pandas as pd
from ultralytics import YOLO
import datetime # Import untuk unique sheet names

# --- KONFIGURASI PATH DAN HYPERPARAMETER ---

# Path ke file data.yaml Anda
DATA_YAML_PATH = "/kaggle/working/data2.yaml"

# Ganti dengan hyperparameter terbaik hasil tuning manual Anda (atau gunakan default)
BEST_LR = 0.005      # Learning Rate awal (Default)
BEST_MOMENTUM = 0.937 # Momentum (Default)
FINAL_EPOCHS = 20   # Jumlah epoch untuk pelatihan final (Ganti sesuai kebutuhan)
BATCH_SIZE = 16        # Ukuran batch (Sesuaikan dengan VRAM GPU Kaggle)

# Output directory di Google Drive untuk semua eksperimen YOLO
# Ini akan membuat folder seperti: /content/drive/MyDrive/EAS VISIKOM/YOLO_Output/yolo_production/YOLOv8_Final_Training_Model2
OUTPUT_BASE_DRIVE_DIR = "/kaggle/working/Hasil"

# Nama proyek (subfolder di OUTPUT_BASE_DRIVE_DIR)
PROJECT_NAME = "yolo_production"

# Nama eksperimen final (output akan disimpan di PROJECT_NAME/EXPERIMENT_NAME)
EXPERIMENT_NAME = 'YOLOv8_Final_Training_Model4'

# --- INIALISASI MODEL ---
# Muat model pre-trained (yolov8n.pt)
model = YOLO('yolov8n.pt')

# ----------------------------------------------------
# Bagian 1: Pelatihan Final
# ----------------------------------------------------
print("---> Memulai Pelatihan Final YOLOv8 --->")


TRAINING_ARGS = {
    'data': DATA_YAML_PATH,
    'epochs': FINAL_EPOCHS,
    'imgsz': 640,
    'batch': BATCH_SIZE,
    'name': EXPERIMENT_NAME, # Experiment name within the project
    'project': os.path.join(OUTPUT_BASE_DRIVE_DIR, PROJECT_NAME), # Full path to the project folder on Drive
    'lr0': BEST_LR,
    'momentum': BEST_MOMENTUM,
    'val': True, # Wajib untuk mendapatkan metrik validasi
    'plots': True # Menghasilkan plot loss dan metrik
}

# Mulai Pelatihan
results = model.train(
    **TRAINING_ARGS
)

print("\n✅ Pelatihan Selesai!")

# ----------------------------------------------------
# Bagian 2: Evaluasi dan Pencatatan Loss
# ----------------------------------------------------

# Tentukan path tempat hasil disimpan (otomatis dibuat oleh Ultralytics)
# Ultralytics menyimpan hasil relatif terhadap direktori kerja saat ini,
# biasanya di `runs/detect/project_name/experiment_name`.
# Objek `results` secara langsung menyediakan direktori penyimpanan.
OUTPUT_BASE_DIR = str(results.save_dir) # This will now be the Drive path
BEST_MODEL_PATH = os.path.join(OUTPUT_BASE_DIR, 'weights', 'best.pt')
RESULTS_CSV_PATH = os.path.join(OUTPUT_BASE_DIR, 'results.csv')

# Path untuk menyimpan hasil ke file Excel
# The Excel report should also go into the new OUTPUT_BASE_DIR
EXCEL_OUTPUT_DIR = os.path.join(OUTPUT_BASE_DIR, 'Excel_Reports') # Changed this line
os.makedirs(EXCEL_OUTPUT_DIR, exist_ok=True)
EXCEL_REPORT_PATH = os.path.join(EXCEL_OUTPUT_DIR, f'{PROJECT_NAME}_{EXPERIMENT_NAME}_training_results.xlsx') # Added experiment name to differentiate reports

print("\n--- Ringkasan Hasil Pelatihan ---")

# A. Mengambil Metrik mAP (Mean Average Precision)
try:
    # Akses metrik langsung dari objek results.metrics (DetMetrics object)
    map50 = results.metrics.box.map50
    map50_95 = results.metrics.box.map

    print(f"Metrik Deteksi (Bounding Box - B):")
    print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
    print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")

except Exception as e:
    print(f"Terjadi error saat mencoba mengakses mAP dari results.metrics: {e}. Mencoba membaca dari CSV...")
    # Fallback to reading from CSV if direct access fails or for consistency
    try:
        df_results_full = pd.read_csv(RESULTS_CSV_PATH)
        last_row_full = df_results_full.iloc[-1]
        map50 = last_row_full['metrics/mAP50(B)']
        map50_95 = last_row_full['metrics/mAP50-95(B)']
        print(f"Metrik Deteksi (Bounding Box - B) dari CSV:")
        print(f"  mAP50 (Akurasi di IoU 50%): {map50:.4f}")
        print(f"  mAP50-95 (Akurasi Rata-rata): {map50_95:.4f}")
    except Exception as csv_e:
        print(f"❌ ERROR membaca mAP dari CSV: {csv_e}")

# B. Mengambil Nilai Loss dari File results.csv dan menyimpannya ke Excel
try:
    # Membaca file results.csv yang dibuat oleh Ultralytics
    df_results = pd.read_csv(RESULTS_CSV_PATH)

    # Mencari baris terakhir (epoch terakhir)
    last_row = df_results.iloc[-1]

    # Kolom Loss di Ultralytics:
    # box_loss: Loss untuk bounding box localization
    # cls_loss: Loss untuk classification
    # dfl_loss: Loss untuk Distribution Focal Loss (digunakan di YOLOv8)

    train_box_loss = last_row['train/box_loss']
    train_cls_loss = last_row['train/cls_loss']
    val_box_loss = last_row['val/box_loss']
    val_cls_loss = last_row['val/cls_loss']

    print("\nNilai Loss di Akhir Pelatihan:")
    print(f"  Total Train Loss (Box+Cls): {(train_box_loss + train_cls_loss):.4f}")
    print(f"  Total Validasi Loss (Box+Cls): {(val_box_loss + val_cls_loss):.4f}")
    print(f"  (Detail: Train Box Loss: {train_box_loss:.4f} | Val Box Loss: {val_box_loss:.4f})")

    # --- Menyimpan ke Excel/Spreadsheet --- #
    # Buat nama sheet yang unik berdasarkan waktu
    timestamp_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    sheet_name = f"Run_{timestamp_str}"

    # Gunakan pd.ExcelWriter untuk menulis ke file Excel.
    # 'mode='a'' untuk append jika file sudah ada.
    # 'if_sheet_exists='new'' akan membuat sheet baru jika nama sheet sudah ada (atau file belum ada).
    # 'engine='openpyxl'' diperlukan untuk mode append dan .xlsx.
    with pd.ExcelWriter(EXCEL_REPORT_PATH, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df_results.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Hasil pelatihan disimpan ke sheet '{sheet_name}' di file Excel: {EXCEL_REPORT_PATH}")


except FileNotFoundError:
    print(f"\n❌ ERROR: File results.csv tidak ditemukan di {RESULTS_CSV_PATH}.")
    print("Pastikan eksperimen berjalan sepenuhnya dan tidak ada error file system.")
except Exception as e:
    print(f"\n❌ ERROR membaca Loss dari CSV atau menulis ke Excel: {e}")

# ----------------------------------------------------
# Bagian 3: Konfirmasi Penyimpanan Model
# ----------------------------------------------------
print("\n--- Konfirmasi Penyimpanan Model ---")
print(f"Model terbaik tersimpan secara otomatis di:")
print(f"💾 {BEST_MODEL_PATH}")
print(f"Log pelatihan (loss dan metrik) tersimpan di:")
print(f"📄 {RESULTS_CSV_PATH}")
print(f"Laporan Excel tersimpan di:")
print(f"📊 {EXCEL_REPORT_PATH}")


# ViTTT

Traing Model ViT, beberapa cell berisikan hyperparameter yang berbeda-beda. Lokasi penyimpanan hasil dapat disesuaikan kembali.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. KONFIGURASI DAN PATHS ---
DATA_DIR = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug'
OUTPUT_DIR = '/kaggle/working/HasilViT_Corn/p1'
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, 'vision_transformer_corn_disease_best.pth')
EXCEL_REPORT_PATH = os.path.join(OUTPUT_DIR, 'vit_training_evaluation_report.xlsx')
CONF_MATRIX_PATH = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
TRAINING_PLOT_PATH = os.path.join(OUTPUT_DIR, 'training_loss_accuracy_plot.png')

# Hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
VALIDATION_SPLIT = 0.2
# Koreksi: Mengatur NUM_WORKERS ke 0 untuk stabilitas di lingkungan Notebook/Kaggle
NUM_WORKERS = 0 

# Otomatis hitung jumlah kelas
try:
    NUM_CLASSES = len(os.listdir(DATA_DIR))
except FileNotFoundError:
    print(f"ERROR: Data directory not found at {DATA_DIR}")
    NUM_CLASSES = 0 

# Pastikan folder output ada
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Semua output akan disimpan di: {OUTPUT_DIR}")

# --- 2. PREPROCESSING DAN AUGMENTASI KHUSUS VIT ---
# Transformasi Gambar (Standar ViT/ImageNet)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- 3. PEMBAGIAN DATA (TRAIN DAN VALIDATION) ---

print("--- Memuat dan Membagi Data ---")

# Memuat seluruh dataset dari folder
full_dataset = datasets.ImageFolder(DATA_DIR, data_transforms['val']) 
class_names = full_dataset.classes
print(f"Total Kelas: {class_names} ({NUM_CLASSES} kelas)")

# Tentukan ukuran data train dan validasi
val_size = int(VALIDATION_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size

# Pembagian acak
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Terapkan transformasi yang benar secara manual
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val'] 

# Buat DataLoader
# KOREKSI: Menggunakan NUM_WORKERS = 0 untuk menghindari masalah multiprocessing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Tentukan Device (GPU jika tersedia)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan perangkat: {device}")

# --- 4. INISIALISASI MODEL VIT ---
# Fungsi untuk membuat model ViT (berguna saat loading model di Langkah 6)
def initialize_vit_model(num_classes):
    model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
    # Mengganti Classification Head
    num_ftrs = model.heads.head.in_features
    model.heads.head = nn.Linear(num_ftrs, num_classes)
    return model

model = initialize_vit_model(NUM_CLASSES)
print("Vision Transformer (ViT-Base) berhasil dimuat.")

model = model.to(device)

# Definisi Fungsi Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- FUNGSI BARU UNTUK PLOTTING (SUDAH DIKOREKSI DARI NAMEERROR) ---
def plot_training_history(history, save_path):
    """Membuat plot Loss dan Accuracy dari histori training."""
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    val_acc_percent = [acc * 100 for acc in history['val_acc']] 
    plt.plot(epochs, val_acc_percent, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Plot Training History tersimpan di: {save_path}") 

# --- 5. FUNGSI TRAINING DAN VALIDASI ---

def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):
    start_time = time.time()
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []} 

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"Model tersimpan di {MODEL_SAVE_PATH} (Acc: {best_acc:.4f})")

    time_elapsed = time.time() - start_time
    print(f'\nTraining Selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Val Acc: {best_acc:.4f}')

    return model, history

# Lakukan Training
model_ft, training_history = train_model(model, criterion, optimizer)

# Visualisasikan History Training
plot_training_history(training_history, TRAINING_PLOT_PATH)


# --- 6. EVALUASI AKHIR, CONFUSION MATRIX, DAN PENYIMPANAN METRIK KE EXCEL ---

def evaluate_and_save_report(model_architecture, loader, class_names, history, conf_matrix_path, excel_report_path):
    
    # KOREKSI: Inisialisasi model di sini sebelum loading state_dict
    model_eval = initialize_vit_model(len(class_names))
    model_eval.load_state_dict(torch.load(MODEL_SAVE_PATH))
    model_eval = model_eval.to(device)
    model_eval.eval()

    all_preds = []
    all_labels = []

    print("\n--- Evaluasi Model Terbaik pada Data Validasi ---")
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_eval(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- A. Classification Report (Metrik) ---
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    df_report.index.name = 'Metric'
    print("\nClassification Report (Data Validasi):")
    print(df_report)

    # --- B. Confusion Matrix (Metrik Konvolusi) ---
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(conf_matrix_path)
    print(f"\nConfusion Matrix tersimpan di: {conf_matrix_path}")
    
    
    # --- C. Simpan ke Excel ---
    df_history = pd.DataFrame({
        'Epoch': range(1, len(history['train_loss']) + 1),
        'Train Loss': history['train_loss'],
        'Val Loss': history['val_loss'],
        'Train Accuracy': history['train_acc'], 
        'Val Accuracy': history['val_acc']
    })

    with pd.ExcelWriter(excel_report_path) as writer:
        df_report.to_excel(writer, sheet_name='Evaluation_Metrics')
        df_history.to_excel(writer, sheet_name='Training_History', index=False)

    print(f"\nLaporan Evaluasi dan Histori Training tersimpan di: {excel_report_path}")
    print("Selesai. Semua output berada di folder:", os.path.dirname(excel_report_path))

# Lakukan Evaluasi dan Penyimpanan
# Perhatikan bahwa kita menggunakan fungsi initialize_vit_model sebagai argumen placeholder
evaluate_and_save_report(initialize_vit_model, 
                         val_loader, 
                         class_names, 
                         training_history, 
                         CONF_MATRIX_PATH, 
                         EXCEL_REPORT_PATH)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. KONFIGURASI DAN PATHS ---
DATA_DIR = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug'
OUTPUT_DIR = '/kaggle/working/HasilViT_Corn/p2'
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, 'vision_transformer_corn_disease_best.pth')
EXCEL_REPORT_PATH = os.path.join(OUTPUT_DIR, 'vit_training_evaluation_report.xlsx')
CONF_MATRIX_PATH = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
TRAINING_PLOT_PATH = os.path.join(OUTPUT_DIR, 'training_loss_accuracy_plot.png')

# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
VALIDATION_SPLIT = 0.2
# Koreksi: Mengatur NUM_WORKERS ke 0 untuk stabilitas di lingkungan Notebook/Kaggle
NUM_WORKERS = 0 

# Otomatis hitung jumlah kelas
try:
    NUM_CLASSES = len(os.listdir(DATA_DIR))
except FileNotFoundError:
    print(f"ERROR: Data directory not found at {DATA_DIR}")
    NUM_CLASSES = 0 

# Pastikan folder output ada
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Semua output akan disimpan di: {OUTPUT_DIR}")

# --- 2. PREPROCESSING DAN AUGMENTASI KHUSUS VIT ---
# Transformasi Gambar (Standar ViT/ImageNet)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- 3. PEMBAGIAN DATA (TRAIN DAN VALIDATION) ---

print("--- Memuat dan Membagi Data ---")

# Memuat seluruh dataset dari folder
full_dataset = datasets.ImageFolder(DATA_DIR, data_transforms['val']) 
class_names = full_dataset.classes
print(f"Total Kelas: {class_names} ({NUM_CLASSES} kelas)")

# Tentukan ukuran data train dan validasi
val_size = int(VALIDATION_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size

# Pembagian acak
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Terapkan transformasi yang benar secara manual
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val'] 

# Buat DataLoader
# KOREKSI: Menggunakan NUM_WORKERS = 0 untuk menghindari masalah multiprocessing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Tentukan Device (GPU jika tersedia)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan perangkat: {device}")

# --- 4. INISIALISASI MODEL VIT ---
# Fungsi untuk membuat model ViT (berguna saat loading model di Langkah 6)
def initialize_vit_model(num_classes):
    model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
    # Mengganti Classification Head
    num_ftrs = model.heads.head.in_features
    model.heads.head = nn.Linear(num_ftrs, num_classes)
    return model

model = initialize_vit_model(NUM_CLASSES)
print("Vision Transformer (ViT-Base) berhasil dimuat.")

model = model.to(device)

# Definisi Fungsi Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- FUNGSI BARU UNTUK PLOTTING (SUDAH DIKOREKSI DARI NAMEERROR) ---
def plot_training_history(history, save_path):
    """Membuat plot Loss dan Accuracy dari histori training."""
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    val_acc_percent = [acc * 100 for acc in history['val_acc']] 
    plt.plot(epochs, val_acc_percent, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Plot Training History tersimpan di: {save_path}") 

# --- 5. FUNGSI TRAINING DAN VALIDASI ---

def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):
    start_time = time.time()
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []} 

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"Model tersimpan di {MODEL_SAVE_PATH} (Acc: {best_acc:.4f})")

    time_elapsed = time.time() - start_time
    print(f'\nTraining Selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Val Acc: {best_acc:.4f}')

    return model, history

# Lakukan Training
model_ft, training_history = train_model(model, criterion, optimizer)

# Visualisasikan History Training
plot_training_history(training_history, TRAINING_PLOT_PATH)


# --- 6. EVALUASI AKHIR, CONFUSION MATRIX, DAN PENYIMPANAN METRIK KE EXCEL ---

def evaluate_and_save_report(model_architecture, loader, class_names, history, conf_matrix_path, excel_report_path):
    
    # KOREKSI: Inisialisasi model di sini sebelum loading state_dict
    model_eval = initialize_vit_model(len(class_names))
    model_eval.load_state_dict(torch.load(MODEL_SAVE_PATH))
    model_eval = model_eval.to(device)
    model_eval.eval()

    all_preds = []
    all_labels = []

    print("\n--- Evaluasi Model Terbaik pada Data Validasi ---")
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_eval(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- A. Classification Report (Metrik) ---
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    df_report.index.name = 'Metric'
    print("\nClassification Report (Data Validasi):")
    print(df_report)

    # --- B. Confusion Matrix (Metrik Konvolusi) ---
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(conf_matrix_path)
    print(f"\nConfusion Matrix tersimpan di: {conf_matrix_path}")
    
    
    # --- C. Simpan ke Excel ---
    df_history = pd.DataFrame({
        'Epoch': range(1, len(history['train_loss']) + 1),
        'Train Loss': history['train_loss'],
        'Val Loss': history['val_loss'],
        'Train Accuracy': history['train_acc'], 
        'Val Accuracy': history['val_acc']
    })

    with pd.ExcelWriter(excel_report_path) as writer:
        df_report.to_excel(writer, sheet_name='Evaluation_Metrics')
        df_history.to_excel(writer, sheet_name='Training_History', index=False)

    print(f"\nLaporan Evaluasi dan Histori Training tersimpan di: {excel_report_path}")
    print("Selesai. Semua output berada di folder:", os.path.dirname(excel_report_path))

# Lakukan Evaluasi dan Penyimpanan
# Perhatikan bahwa kita menggunakan fungsi initialize_vit_model sebagai argumen placeholder
evaluate_and_save_report(initialize_vit_model, 
                         val_loader, 
                         class_names, 
                         training_history, 
                         CONF_MATRIX_PATH, 
                         EXCEL_REPORT_PATH)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. KONFIGURASI DAN PATHS ---
DATA_DIR = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug'
OUTPUT_DIR = '/kaggle/working/HasilViT_Corn/p3'
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, 'vision_transformer_corn_disease_best.pth')
EXCEL_REPORT_PATH = os.path.join(OUTPUT_DIR, 'vit_training_evaluation_report.xlsx')
CONF_MATRIX_PATH = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
TRAINING_PLOT_PATH = os.path.join(OUTPUT_DIR, 'training_loss_accuracy_plot.png')

# Hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 5e-5
VALIDATION_SPLIT = 0.2
# Koreksi: Mengatur NUM_WORKERS ke 0 untuk stabilitas di lingkungan Notebook/Kaggle
NUM_WORKERS = 0 

# Otomatis hitung jumlah kelas
try:
    NUM_CLASSES = len(os.listdir(DATA_DIR))
except FileNotFoundError:
    print(f"ERROR: Data directory not found at {DATA_DIR}")
    NUM_CLASSES = 0 

# Pastikan folder output ada
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Semua output akan disimpan di: {OUTPUT_DIR}")

# --- 2. PREPROCESSING DAN AUGMENTASI KHUSUS VIT ---
# Transformasi Gambar (Standar ViT/ImageNet)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- 3. PEMBAGIAN DATA (TRAIN DAN VALIDATION) ---

print("--- Memuat dan Membagi Data ---")

# Memuat seluruh dataset dari folder
full_dataset = datasets.ImageFolder(DATA_DIR, data_transforms['val']) 
class_names = full_dataset.classes
print(f"Total Kelas: {class_names} ({NUM_CLASSES} kelas)")

# Tentukan ukuran data train dan validasi
val_size = int(VALIDATION_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size

# Pembagian acak
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Terapkan transformasi yang benar secara manual
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val'] 

# Buat DataLoader
# KOREKSI: Menggunakan NUM_WORKERS = 0 untuk menghindari masalah multiprocessing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Tentukan Device (GPU jika tersedia)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan perangkat: {device}")

# --- 4. INISIALISASI MODEL VIT ---
# Fungsi untuk membuat model ViT (berguna saat loading model di Langkah 6)
def initialize_vit_model(num_classes):
    model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
    # Mengganti Classification Head
    num_ftrs = model.heads.head.in_features
    model.heads.head = nn.Linear(num_ftrs, num_classes)
    return model

model = initialize_vit_model(NUM_CLASSES)
print("Vision Transformer (ViT-Base) berhasil dimuat.")

model = model.to(device)

# Definisi Fungsi Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- FUNGSI BARU UNTUK PLOTTING (SUDAH DIKOREKSI DARI NAMEERROR) ---
def plot_training_history(history, save_path):
    """Membuat plot Loss dan Accuracy dari histori training."""
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    val_acc_percent = [acc * 100 for acc in history['val_acc']] 
    plt.plot(epochs, val_acc_percent, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Plot Training History tersimpan di: {save_path}") 

# --- 5. FUNGSI TRAINING DAN VALIDASI ---

def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):
    start_time = time.time()
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []} 

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"Model tersimpan di {MODEL_SAVE_PATH} (Acc: {best_acc:.4f})")

    time_elapsed = time.time() - start_time
    print(f'\nTraining Selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Val Acc: {best_acc:.4f}')

    return model, history

# Lakukan Training
model_ft, training_history = train_model(model, criterion, optimizer)

# Visualisasikan History Training
plot_training_history(training_history, TRAINING_PLOT_PATH)


# --- 6. EVALUASI AKHIR, CONFUSION MATRIX, DAN PENYIMPANAN METRIK KE EXCEL ---

def evaluate_and_save_report(model_architecture, loader, class_names, history, conf_matrix_path, excel_report_path):
    
    # KOREKSI: Inisialisasi model di sini sebelum loading state_dict
    model_eval = initialize_vit_model(len(class_names))
    model_eval.load_state_dict(torch.load(MODEL_SAVE_PATH))
    model_eval = model_eval.to(device)
    model_eval.eval()

    all_preds = []
    all_labels = []

    print("\n--- Evaluasi Model Terbaik pada Data Validasi ---")
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_eval(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- A. Classification Report (Metrik) ---
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    df_report.index.name = 'Metric'
    print("\nClassification Report (Data Validasi):")
    print(df_report)

    # --- B. Confusion Matrix (Metrik Konvolusi) ---
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(conf_matrix_path)
    print(f"\nConfusion Matrix tersimpan di: {conf_matrix_path}")
    
    
    # --- C. Simpan ke Excel ---
    df_history = pd.DataFrame({
        'Epoch': range(1, len(history['train_loss']) + 1),
        'Train Loss': history['train_loss'],
        'Val Loss': history['val_loss'],
        'Train Accuracy': history['train_acc'], 
        'Val Accuracy': history['val_acc']
    })

    with pd.ExcelWriter(excel_report_path) as writer:
        df_report.to_excel(writer, sheet_name='Evaluation_Metrics')
        df_history.to_excel(writer, sheet_name='Training_History', index=False)

    print(f"\nLaporan Evaluasi dan Histori Training tersimpan di: {excel_report_path}")
    print("Selesai. Semua output berada di folder:", os.path.dirname(excel_report_path))

# Lakukan Evaluasi dan Penyimpanan
# Perhatikan bahwa kita menggunakan fungsi initialize_vit_model sebagai argumen placeholder
evaluate_and_save_report(initialize_vit_model, 
                         val_loader, 
                         class_names, 
                         training_history, 
                         CONF_MATRIX_PATH, 
                         EXCEL_REPORT_PATH)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. KONFIGURASI DAN PATHS ---
DATA_DIR = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug'
OUTPUT_DIR = '/kaggle/working/HasilViT_Corn/p4'
MODEL_SAVE_PATH = os.path.join(OUTPUT_DIR, 'vision_transformer_corn_disease_best.pth')
EXCEL_REPORT_PATH = os.path.join(OUTPUT_DIR, 'vit_training_evaluation_report.xlsx')
CONF_MATRIX_PATH = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
TRAINING_PLOT_PATH = os.path.join(OUTPUT_DIR, 'training_loss_accuracy_plot.png')

# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 5e-5
VALIDATION_SPLIT = 0.2
# Koreksi: Mengatur NUM_WORKERS ke 0 untuk stabilitas di lingkungan Notebook/Kaggle
NUM_WORKERS = 0 

# Otomatis hitung jumlah kelas
try:
    NUM_CLASSES = len(os.listdir(DATA_DIR))
except FileNotFoundError:
    print(f"ERROR: Data directory not found at {DATA_DIR}")
    NUM_CLASSES = 0 

# Pastikan folder output ada
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Semua output akan disimpan di: {OUTPUT_DIR}")

# --- 2. PREPROCESSING DAN AUGMENTASI KHUSUS VIT ---
# Transformasi Gambar (Standar ViT/ImageNet)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- 3. PEMBAGIAN DATA (TRAIN DAN VALIDATION) ---

print("--- Memuat dan Membagi Data ---")

# Memuat seluruh dataset dari folder
full_dataset = datasets.ImageFolder(DATA_DIR, data_transforms['val']) 
class_names = full_dataset.classes
print(f"Total Kelas: {class_names} ({NUM_CLASSES} kelas)")

# Tentukan ukuran data train dan validasi
val_size = int(VALIDATION_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size

# Pembagian acak
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Terapkan transformasi yang benar secara manual
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val'] 

# Buat DataLoader
# KOREKSI: Menggunakan NUM_WORKERS = 0 untuk menghindari masalah multiprocessing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Tentukan Device (GPU jika tersedia)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan perangkat: {device}")

# --- 4. INISIALISASI MODEL VIT ---
# Fungsi untuk membuat model ViT (berguna saat loading model di Langkah 6)
def initialize_vit_model(num_classes):
    model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
    # Mengganti Classification Head
    num_ftrs = model.heads.head.in_features
    model.heads.head = nn.Linear(num_ftrs, num_classes)
    return model

model = initialize_vit_model(NUM_CLASSES)
print("Vision Transformer (ViT-Base) berhasil dimuat.")

model = model.to(device)

# Definisi Fungsi Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- FUNGSI BARU UNTUK PLOTTING (SUDAH DIKOREKSI DARI NAMEERROR) ---
def plot_training_history(history, save_path):
    """Membuat plot Loss dan Accuracy dari histori training."""
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    val_acc_percent = [acc * 100 for acc in history['val_acc']] 
    plt.plot(epochs, val_acc_percent, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Plot Training History tersimpan di: {save_path}") 

# --- 5. FUNGSI TRAINING DAN VALIDASI ---

def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):
    start_time = time.time()
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []} 

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"Model tersimpan di {MODEL_SAVE_PATH} (Acc: {best_acc:.4f})")

    time_elapsed = time.time() - start_time
    print(f'\nTraining Selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Val Acc: {best_acc:.4f}')

    return model, history

# Lakukan Training
model_ft, training_history = train_model(model, criterion, optimizer)

# Visualisasikan History Training
plot_training_history(training_history, TRAINING_PLOT_PATH)


# --- 6. EVALUASI AKHIR, CONFUSION MATRIX, DAN PENYIMPANAN METRIK KE EXCEL ---

def evaluate_and_save_report(model_architecture, loader, class_names, history, conf_matrix_path, excel_report_path):
    
    # KOREKSI: Inisialisasi model di sini sebelum loading state_dict
    model_eval = initialize_vit_model(len(class_names))
    model_eval.load_state_dict(torch.load(MODEL_SAVE_PATH))
    model_eval = model_eval.to(device)
    model_eval.eval()

    all_preds = []
    all_labels = []

    print("\n--- Evaluasi Model Terbaik pada Data Validasi ---")
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_eval(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # --- A. Classification Report (Metrik) ---
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    df_report.index.name = 'Metric'
    print("\nClassification Report (Data Validasi):")
    print(df_report)

    # --- B. Confusion Matrix (Metrik Konvolusi) ---
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(conf_matrix_path)
    print(f"\nConfusion Matrix tersimpan di: {conf_matrix_path}")
    
    
    # --- C. Simpan ke Excel ---
    df_history = pd.DataFrame({
        'Epoch': range(1, len(history['train_loss']) + 1),
        'Train Loss': history['train_loss'],
        'Val Loss': history['val_loss'],
        'Train Accuracy': history['train_acc'], 
        'Val Accuracy': history['val_acc']
    })

    with pd.ExcelWriter(excel_report_path) as writer:
        df_report.to_excel(writer, sheet_name='Evaluation_Metrics')
        df_history.to_excel(writer, sheet_name='Training_History', index=False)

    print(f"\nLaporan Evaluasi dan Histori Training tersimpan di: {excel_report_path}")
    print("Selesai. Semua output berada di folder:", os.path.dirname(excel_report_path))

# Lakukan Evaluasi dan Penyimpanan
# Perhatikan bahwa kita menggunakan fungsi initialize_vit_model sebagai argumen placeholder
evaluate_and_save_report(initialize_vit_model, 
                         val_loader, 
                         class_names, 
                         training_history, 
                         CONF_MATRIX_PATH, 
                         EXCEL_REPORT_PATH)

# CNN

Traing Model CNN, beberapa cell berisikan hyperparameter yang berbeda-beda. Lokasi penyimpanan hasil dapat disesuaikan kembali.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

# 1. KONFIGURASI & PERSIAPAN FOLDER
DATASET_PATH = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug' # Ganti dengan path dataset Anda
OUTPUT_DIR = '/kaggle/working/HasilCNN2'   # Nama folder output
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 64
RANDOM_SEED = 123

# Membuat folder output jika belum ada
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Folder '{OUTPUT_DIR}' berhasil dibuat.")

# 2. LOAD DATASET
# Train Split (80%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Val Split (20%) - Untuk Validasi saat Training
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Simpan nama kelas
class_names = train_ds.class_names
num_classes = len(class_names)

# Optimasi dataset untuk training
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3. MEMBANGUN MODEL
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. CALLBACK UNTUK MENYIMPAN MODEL TERBAIK
# Model hanya akan disimpan jika 'val_accuracy' meningkat
checkpoint_path = os.path.join(OUTPUT_DIR, 'model_terbaik.keras')
model_checkpoint = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# 5. TRAINING
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint] # Tambahkan callback di sini
)

# 6. SIMPAN PLOT GRAFIK KE FOLDER
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 5))

# Plot Akurasi
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

# Simpan gambar
plot_path = os.path.join(OUTPUT_DIR, 'grafik_akurasi_loss.png')
plt.savefig(plot_path)
print(f"Grafik disimpan di: {plot_path}")
plt.close() # Tutup plot agar tidak memakan memori

# 7. EVALUASI METRIK LENGKAP & SIMPAN KE EXCEL
print("Sedang menghitung metrik evaluasi...")

# Kita perlu meload ulang data validasi TANPA shuffle agar urutan prediksi sesuai label
val_ds_eval = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False # PENTING: Jangan di-shuffle untuk evaluasi
)

# Ambil label asli (y_true) dan prediksi (y_pred)
y_true = []
y_pred = []

# Loop batch untuk prediksi
for images, labels in val_ds_eval:
    y_true.extend(labels.numpy())
    predictions = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(predictions, axis=1))

# Buat Report menggunakan Scikit-Learn
report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Konversi ke Pandas DataFrame
df_report = pd.DataFrame(report_dict).transpose()

# Simpan ke Excel
excel_path = os.path.join(OUTPUT_DIR, 'laporan_evaluasi.xlsx')
df_report.to_excel(excel_path)

print(f"File Excel evaluasi disimpan di: {excel_path}")
print("Selesai.")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

# 1. KONFIGURASI & PERSIAPAN FOLDER
DATASET_PATH = '/kaggle/input/crop-augmentation-citra-jagung/Data Crop_aug' # Ganti dengan path dataset Anda
OUTPUT_DIR = '/kaggle/working/HasilCNN'   # Nama folder output
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
RANDOM_SEED = 123

# Membuat folder output jika belum ada
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Folder '{OUTPUT_DIR}' berhasil dibuat.")

# 2. LOAD DATASET
# Train Split (80%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Val Split (20%) - Untuk Validasi saat Training
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Simpan nama kelas
class_names = train_ds.class_names
num_classes = len(class_names)

# Optimasi dataset untuk training
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3. MEMBANGUN MODEL
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. CALLBACK UNTUK MENYIMPAN MODEL TERBAIK
# Model hanya akan disimpan jika 'val_accuracy' meningkat
checkpoint_path = os.path.join(OUTPUT_DIR, 'model_terbaik.keras')
model_checkpoint = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# 5. TRAINING
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint] # Tambahkan callback di sini
)

# 6. SIMPAN PLOT GRAFIK KE FOLDER
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 5))

# Plot Akurasi
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

# Simpan gambar
plot_path = os.path.join(OUTPUT_DIR, 'grafik_akurasi_loss.png')
plt.savefig(plot_path)
print(f"Grafik disimpan di: {plot_path}")
plt.close() # Tutup plot agar tidak memakan memori

# 7. EVALUASI METRIK LENGKAP & SIMPAN KE EXCEL
print("Sedang menghitung metrik evaluasi...")

# Kita perlu meload ulang data validasi TANPA shuffle agar urutan prediksi sesuai label
val_ds_eval = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False # PENTING: Jangan di-shuffle untuk evaluasi
)

# Ambil label asli (y_true) dan prediksi (y_pred)
y_true = []
y_pred = []

# Loop batch untuk prediksi
for images, labels in val_ds_eval:
    y_true.extend(labels.numpy())
    predictions = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(predictions, axis=1))

# Buat Report menggunakan Scikit-Learn
report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Konversi ke Pandas DataFrame
df_report = pd.DataFrame(report_dict).transpose()

# Simpan ke Excel
excel_path = os.path.join(OUTPUT_DIR, 'laporan_evaluasi.xlsx')
df_report.to_excel(excel_path)

print(f"File Excel evaluasi disimpan di: {excel_path}")
print("Selesai.")